In [ ]:
import matplotlib.pyplot as plt
import cv2
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
sample_cat = cv2.imread('./DATA/CATS_DOGS/train/CAT/4.jpg')
sample_cat = cv2.cvtColor(sample_cat, cv2.COLOR_BGR2RGB)

plt.imshow(sample_cat)

In [ ]:
sample_dog = cv2.imread('./DATA/CATS_DOGS/train/DOG/4.jpg')

sample_dog = cv2.cvtColor(sample_dog, cv2.COLOR_BGR2RGB)

plt.imshow(sample_dog)

In [ ]:
image_gen = ImageDataGenerator(rotation_range=30, # rotate the image 30 degrees
                               width_shift_range=0.1, # Shift the pic width by a max of 10%
                               height_shift_range=0.1, # Shift the pic height by a max of 10%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.2, # Shear means cutting away part of the image (max 20%)
                               zoom_range=0.2, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

In [ ]:
plt.imshow(image_gen.random_transform(sample_dog))

In [ ]:
image_gen.flow_from_directory('./DATA/CATS_DOGS/train')

In [ ]:
image_gen.flow_from_directory('./DATA/CATS_DOGS/test')

In [ ]:
from keras import callbacks
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.models import Sequential

In [ ]:
image_shape = (150,150,3)

model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=image_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=image_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=image_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



In [ ]:
model.summary()

In [ ]:
batch_size = 16

train_image_gen = image_gen.flow_from_directory('./DATA/CATS_DOGS/train',
                                               target_size=image_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='binary')

In [ ]:
test_image_gen = image_gen.flow_from_directory('./DATA/CATS_DOGS/test',
                                               target_size=image_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='binary')

In [ ]:
train_image_gen.class_indices

In [ ]:
results = model.fit_generator(train_image_gen,epochs=100,
                              steps_per_epoch=150,
                              validation_data=test_image_gen,
                             validation_steps=12)

In [ ]:
model.save('CAT_DOG_CNN_keras.h5')

In [ ]:
results.history['acc']

In [ ]:
plt.plot(results.history['acc'])

In [ ]:
train_image_gen.class_indices

In [ ]:
import numpy as np
from keras.preprocessing import image

dog_file = '../DATA/CATS_DOGS/train/Dog/2.jpg'

dog_img = image.load_img(dog_file, target_size=(150, 150))

dog_img = image.img_to_array(dog_img)

dog_img = np.expand_dims(dog_img, axis=0)
dog_img = dog_img/255

In [ ]:
prediction_prob = model.predict(dog_img)

In [ ]:
# Output prediction
print(f'Probability that image is a dog is: {prediction_prob} ')